Semantic Segmentation

In [ ]:
! git clone https://github.com/sunghoonYoon/Kitti_TA

from google.colab import drive
drive.mount('/content/drive')

####NOTE: you should change the "runtime option to use GPU"

In [ ]:
# Kitti_options = NONE #SS: Semantic Segmentation , SM: Stereo Matching, OF: Optical flow /// Only these three options are available

# assert Kitti_options in ['SS','SM','OF']

from PIL import Image
from torchvision import transforms
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch
import numpy as np

from collections import OrderedDict
import time
from imageio import imread, imwrite

########
from Kitti_TA.kitti_seg import decode_segmap, denorm, YOUR_IMPLEMENTATION ##import segmentation network (deeplabWplus)
from Kitti_TA.kitti_stereo import YOUR_IMPLEMENTATION #import stereo matching network
from Kitti_TA.kitti_flow import ArrayToTensor, flow2rgb, YOUR_IMPLEMENTATION #import stereo matching network


left0_image_path = '/content/drive/MyDrive/TA_stereo/2011_09_26_drive_0096_sync/image_02/data/0000000000.png'
right0_image_path = '/content/drive/MyDrive/TA_stereo/2011_09_26_drive_0096_sync/image_03/data/0000000000.png'

left1_image_path = '/content/drive/MyDrive/TA_stereo/2011_09_26_drive_0096_sync/image_02/data/0000000001.png'




Semantic Segmentation

In [ ]:

net_SS = YOUR_IMPLEMENTATION.cuda() # declare Network and the number of class=19
state_dict= torch.load('/content/drive/MyDrive/TA_segmentation/cityscapes_best.pth')
net_SS = torch.nn.DataParallel(net_SS)
    
net_SS.YOUR_IMPLEMENTATION(state_dict['state_dict'],strict=False) #load the checkpoint(state_dict) 

net_SS.eval()

image= np.asarray(Image.open(left0_image_path)).convert('RGB')

MEAN = [0.45734706, 0.43338275, 0.40058118]
STD = [0.23965294, 0.23532275, 0.2398498]

toTensor = transforms.ToTensor()
normTensor = transforms.Normalize(MEAN,STD)
imageT = normTensor(toTensor(image))

with torch.no_grad():
    
    imageT = F.interpolate(imageT.unsqueeze(0), size=(512,1024),mode='bilinear',align_corners=True)
    YOUR_IMPLEMENTATION # get segmentation results using network

    YOUR_IMPLEMENTATION # Take softmax to the network output in channel dimension
    YOUR_IMPLEMENTATION # Get max indices from the softmax-ed result (use argmax function)

segmap = YOUR_IMPLEMENTATION #use the function that we provide (convert prediction into RBG color map)

###Visualization###
'''
YOUR IMPLEMENTATION
#Visualize the segmentation result and image together with transparency (alpha=0.7) mode. use plt function

''' 

Stereo matching

In [ ]:

max_disp = 192
net_SM = YOUR_IMPLEMENTATION # declare Network and the max disparity is 192
net_SM = net_SM.cuda()
net_SM.eval()

checkpoint = torch.load('/content/drive/MyDrive/TA_stereo/pretrained_model_KITTI2015.tar')
new_state_dict = OrderedDict()
for k, v in checkpoint['state_dict'].items():
    name = k[7:]
    new_state_dict[name] = v    
net_SM.load_state_dict(new_state_dict)    
normal_mean_var = {'mean': [0.485, 0.456, 0.406],
                    'std': [0.229, 0.224, 0.225]}
infer_transform = transforms.Compose([transforms.ToTensor(),
                                        transforms.Normalize(**normal_mean_var)])    


imgL_o = Image.open(left0_image_path).convert('RGB')
imgR_o = Image.open(right0_image_path).convert('RGB')

imgL = infer_transform(imgL_o)
imgR = infer_transform(imgR_o) 

# pad to width and hight to 16 times
if imgL.shape[1] % 16 != 0:
    times = imgL.shape[1]//16       
    top_pad = (times+1)*16 -imgL.shape[1]
else:
    top_pad = 0

if imgL.shape[2] % 16 != 0:
    times = imgL.shape[2]//16                       
    right_pad = (times+1)*16-imgL.shape[2]
else:
    right_pad = 0    

imgL = F.pad(imgL,(0,right_pad, top_pad,0)).unsqueeze(0)
imgR = F.pad(imgR,(0,right_pad, top_pad,0)).unsqueeze(0)

start_time = time.time()

net_SM.eval()

imgL = imgL.cuda()
imgR = imgR.cuda()  
        

with torch.no_grad():
    disp = YOUR_IMPLEMENTATION # get output from the network, input: (imgL,imgR)

    disp = torch.squeeze(disp)
    pred_disp = disp.data.cpu().numpy()


if top_pad !=0 or right_pad != 0:
    img = pred_disp[top_pad:,:-right_pad]
else:
    img = pred_disp

####Visualize####
'''
YOUR IMPLEMENTATION
#Visualize the left image and left disparity using plt function.

'''

    

optical flow

In [ ]:
state_dict = torch.load('/content/drive/MyDrive/TA_opticalflow/flownets_bn_EPE2.459.pth.tar')

net_OF = YOUR_IMPLEMENTATION # declare Network for optical flow
net_OF = net_OF.cuda()
net_OF.eval()

input_transform = transforms.Compose([
    ArrayToTensor(),
    transforms.Normalize(mean=[0,0,0], std=[255,255,255]),
    transforms.Normalize(mean=[0.411,0.432,0.45], std=[1,1,1])
])


img1= input_transform(imread(left0_image_path))
img2= input_transform(imread(left1_image_path))

input_var = YOUR_IMPLEMENTATION # concatenate "img1" and "img2" and expand zero dimentions so that the dimension (1,C,H,W)

net_OF.load_state_dict(state_dict['state_dict'])

bidirectional = True
if bidirectional:
    # feed inverted pair along with normal pair
    inverted_input_var = YOUR_IMPLEMENTATION # concatenate "img2" and "img1" and expand zero dimentions so that the dimension (1,C,H,W), Note that the order (img2,img1) is important
    input_var = torch.cat([input_var, inverted_input_var])

input_var = input_var.cuda()
# compute output
flow_output = YOUR_IMPLEMENTATION #Get optical flow network output (input: input_var)

rgb_flow = flow2rgb(20 * flow_output[0], max_value=20)
rgb_inv_flow = flow2rgb(20 * flow_output[1], max_value=20)
to_save = (rgb_flow * 255).astype(np.uint8).transpose(1,2,0)
to_save_inv = (rgb_inv_flow * 255).astype(np.uint8).transpose(1,2,0)

plt.figure()
plt.title('flow result')
plt.imshow(to_save)
plt.show()

plt.figure()
plt.title('inv flow result')
plt.imshow(to_save_inv)
plt.show()


Saving results

In [ ]:
file_dir = '/content/drive/MyDrive/TA_stereo/2011_09_26_drive_0096_sync/image_02/data/*.png'
save_dir = '/content/drive/MyDrive/TA_segmentation/output'
import glob
import os
import cv2

files = glob.glob(file_dir)

'''
This is manual (incompleted code) for you to convert output into video.
1) extract the output and save it to png.
2) used cv2.VideoWriter function to make video from png image files
3) Make Video for all 3 tasks (Semantic Segmetnation /Stereo Matching/ Optical Flow)

'''

for file in files:
  with torch.no_grad():
    image = np.asarray(Image.open(file).convert('RGB'))
    imageT = normTensor(toTensor(image))
    imageT = F.interpolate(imageT.unsqueeze(0), size=(512,1024),mode='bilinear',align_corners=True)
    out = net(imageT.cuda())

    '''YOUR IMPLEMENTATION''' #Get output 


    plt.savefig(os.path.join(save_dir,os.path.basename(file)))
    plt.close()

### Convert image to video

print(save_dir+'segmentation_video.avi') #change name if needed
out = cv2.VideoWriter(save_dir+"/"+'segmentation_video.avi',cv2.VideoWriter_fourcc(*"MJPG"), 2, (1024,512)) #change name if needed

for i in range(2):
    img = cv2.imread(os.path.join('/content/drive/MyDrive/TA_segmentation/output',"%010d.png"%i))
    print(img.shape)
    out.write(img)

out.release()